# 1차시: Neural Network 기초
#### Import

In [ ]:
!pip install --upgrade git+https://github.com/dAiv-CNU/torchdaiv.git

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchdaiv import datasets
from torchdaiv.lectures.emotion import nn
from torchdaiv.lectures.emotion.util import vocabulary, transforms

from rich.traceback import install
install(show_locals=True)

#### Load Dataset

In [ ]:
# 사전 생성기 초기화, 데이터셋 로드
vocab, to_vocab = vocabulary.vocabulary_creator()
train_dataset = datasets.EmotionDataset("./data", download=False, train=True, transform=to_vocab)
test_dataset = datasets.EmotionDataset("./data", download=False, train=False, transform=to_vocab)

In [ ]:
# 데이터셋 형태 확인
for i, (text, emotion) in zip(range(10), train_dataset):
    print(f"{i}: {text} -> {emotion}")

In [ ]:
# vocab 생성 후 데이터셋을 텐서로 변환
train_dataset.transform(
    transform=transforms.to_tensor(vocabulary=vocab, show_graph=True),
    target_transform=transforms.label_to_tensor
)
test_dataset.transform(
    transform=transforms.to_tensor(vocabulary=vocab, show_graph=True),
    target_transform=transforms.label_to_tensor
)

In [ ]:
# 텐서 크기 조정
max_token_size = 80
train_dataset.transform(transform=transforms.size_to(max_token_size))
test_dataset.transform(transform=transforms.size_to(max_token_size))

In [ ]:
# 변환된 데이터 형태 확인
for i, (text, emotion) in zip(range(10), train_dataset):
    print(f"{i}: {text} -> {emotion}")

#### DataLoader

In [ ]:
batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, shuffle=True)

#### Model Definition

In [ ]:
# 모델 정의
class MyModel(nn.Module):
    def __init__(self, input_max_token: int, hidden_size: int):
        super(MyModel, self).__init__()
        self.input = nn.Linear(input_max_token, hidden_size)
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc2 = nn.Linear(hidden_size // 2, 3)

    def forward(self, x):
        h1 = F.relu(self.input(x))
        h2 = F.relu(self.fc1(h1))
        out = self.fc2(h2)
        return out

In [ ]:
# 하이퍼 파라미터 설정
epochs = 50
learning_rate = 1e-4
hidden_layer_size = 128 * 5

In [ ]:
# 모델 생성 및 데이터 로더 지정
model = MyModel(input_max_token=max_token_size, hidden_size=hidden_layer_size)
model.set_dataloader(train_dataloader, test_dataloader)
model

In [ ]:
# Use CUDA driver if available
if torch.cuda.is_available():
    model.to("cuda", non_blocking=True)
print("Use Device:", model.device.upper())

#### Training

In [ ]:
# Train Dataset에 대해서 학습 시작
model.train(epochs=epochs, optimizer_init=True, lr=learning_rate)

#### Evaluation

In [ ]:
# Test Dataset에 대해서 평가 진행
model.evaluate()

#### Prediction

In [ ]:
# 토큰화 및 텐서 변환 후 직접 입력한 문장 분류
transform_sequence = [
    transforms.to_tensor(vocabulary=vocab, show_graph=False),
    transforms.size_to(max_token_size)
]

In [ ]:
model.pipeline(message="안녕하세요! 반갑습니다!", transform=transform_sequence)

In [ ]:
model.pipeline(message="이건 좀 아닌거 같음", transform=transform_sequence)

In [ ]:
model.pipeline(message="못 생겼음", transform=transform_sequence)

In [ ]:
model.pipeline(message="잘 받았습니다.", transform=transform_sequence)

In [ ]:
model.pipeline(message="잘 모르겠네요...", transform=transform_sequence)

In [ ]:
model.pipeline(message="색상별로 사이즈가 너무 다르네요 반품귀찮아서 그냥 대충 입으려구요  여름에입긴 더울거같네요 봄.가을용입니다", transform=transform_sequence)

In [ ]:
model.pipeline(message="기대를 안했는데  바지가 생각보다  착용감 편하고 가볍네요 근데 한가지 허리밴드부분이 너무  널어서 좀~  쪼금안 덜  넓었으면 좋겠어요", transform=transform_sequence)